In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
filepath = "movies_metadata.csv"


In [ ]:
# Create a dataframe from csv
df = pd.read_csv(filepath, delimiter=',')
# User list comprehension to create a list of lists from Dataframe rows
list_of_rows = [list(row) for row in df.values]
type(list_of_rows)

In [ ]:
#Check cell 3,3 to check 'genre' information..
print(list_of_rows[3][3])

In [ ]:
#Check length of list - matches excel total number of movies.
print(len(list_of_rows))

In [ ]:
#String has to be converted into list of dictionaries
# found eval function on internet...seems to work.

jd = eval(list_of_rows[3][3])
print(jd)

In [ ]:
#Try with another row..
jd2 = eval(list_of_rows[4][3])
print(jd2)

In [ ]:
# add movie id number to what will be the genre dataframe.
df = pd.DataFrame(jd)
df['movie'] = list_of_rows[3][5]
df

In [ ]:
# Combine dataframes using the concat method..testing with just the two dataframes..

df2 = pd.DataFrame(jd2)
df2['movie'] = list_of_rows[4][5]
frames = [df,df2]
df3 = pd.concat(frames, sort=False)
df3

In [ ]:
# Run a loop for all rows in the spreadsheet. 
#   convert the string in column[3] to python list of dictionaries
#   cabsreate new dataframe from genre information in each row
#   add movie id number from column[5] to dataframe
#   append to a list of dataframes to be concatenated outside the loop.
# Concatenate to one dataframe all the genre information with movie id. Reset index.

list_of_genre_dataframes = []
for row in list_of_rows:
    #print(row[3], row[5])
    genre_list = eval(row[3])
    record_df = pd.DataFrame(genre_list)
    record_df['movie_id_number'] = row[5]
    list_of_genre_dataframes.append(record_df)

genre_table = pd.concat(list_of_genre_dataframes, sort=False)
genre_table.reset_index(inplace=True, drop=True)
genre_table.head(30)

In [31]:
# del genre_table['name']
genre_table = genre_table.rename(columns={'index':'rec_key','id':'genre_id', 'movie_id_number':'movie_id'})
genre_table.head(30)
genre_table.reset_index()

,index,genre_id,movie_id
0,0,16.0,862
1,1,35.0,862
2,2,10751.0,862
3,3,12.0,8844
4,4,14.0,8844
...,...,...,...
91101,91101,10751.0,439050
91102,91102,18.0,111109
91103,91103,28.0,67758
91104,91104,18.0,67758


In [32]:
rds_connection_string = "postgres:postgres@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [33]:
engine.table_names()

['movies_data',
 'keywords',
 'cast',
 'crew',
 'production_companies',
 'production_countries',
 'spoken_languages',
 'ratings',
 'genre_movie_junction',
 'genres',
 'premise']

In [34]:
genre_table.to_sql(name='genre_movie_junction', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "rec_key" violates not-null constraint
DETAIL:  Failing row contains (null, 862, 16).

[SQL: INSERT INTO genre_movie_junction (genre_id, movie_id) VALUES (%(genre_id)s, %(movie_id)s)]
[parameters: ({'genre_id': 16.0, 'movie_id': '862'}, {'genre_id': 35.0, 'movie_id': '862'}, {'genre_id': 10751.0, 'movie_id': '862'}, {'genre_id': 12.0, 'movie_id': '8844'}, {'genre_id': 14.0, 'movie_id': '8844'}, {'genre_id': 10751.0, 'movie_id': '8844'}, {'genre_id': 10749.0, 'movie_id': '15602'}, {'genre_id': 35.0, 'movie_id': '15602'}  ... displaying 10 of 91106 total bound parameter sets ...  {'genre_id': 18.0, 'movie_id': '67758'}, {'genre_id': 53.0, 'movie_id': '67758'})]
(Background on this error at: http://sqlalche.me/e/13/gkpj)